# Python feature extraction plugin hands-on

Jubatusの特徴抽出プラグインをPythonで実装できるようになりました。

実装方法は簡単。Pythonで特徴抽出クラスを記述し、設定ファイルからプラグインを読み込むだけです。

分析コードは特に変更の必要ありません。

以下では、時系列特徴抽出プラグイン (自己回帰プラグイン）を用いて学習と予測を行うためのサンプルコードを提供します。

### Step.1 Jubatusサーバの起動

Python特徴抽出プラグインでは、プラグインの.pyファイルが配置してある場所まで`PYTHONPATH`を通す必要があります。

ここでは `$HOME/jubatus/events/hands-on-5th/python-plugin` に配置することにし、`PYTHONPATH`を以下のように通します。
```
$ export PYTHONPATH=$PYTHONPATH:$HOME/jubatus/events/hands-on-5th/python-plugin
```

`PYTYONPATH`が通せたら、以下のコマンドで回帰用のJubatusサーバを起動します。

```
$ jubaregression -f config.json -t 1000
```

-t オプションはタイムアウト時間で、JupyterからJubatusを呼んだ際の遅延を考慮して設定します。

これで準備は完了です。

### Step.2 関連ライブラリの読み込み

In [ ]:
%matplotlib inline
import datetime
import numpy as np
import pandas as pd
import statsmodels.api as sm # 時系列データ読み込み用
from jubatus import Regression
from jubatus.common import Datum

### Step.3 学習/予測を行うコードの記述

In [ ]:
def train_and_estimate(data, train_rate=0.9, disp_rate=1):
    # 回帰クライアントの起動
    regression = Regression('127.0.0.1', 9199, "")
    regression.clear()

    # 学習用データサイズを設定
    n_train = int(len(data)*train_rate)

    # 可視化用
    answer = []
    prediction = []
    index = df.index[1:]

    for i in range(len(data)-1):
        y_true = float(data[i+1])  # 正解データ
        answer.append(y_true)      # 可視化用

        if i > n_train:
            # 予測を行う
            x = Datum({"data": prediction[-1]})   # 1期前のデータを入力にする
            y_pred = regression.estimate([x])[0]  # 現在の値を予測する
            prediction.append(y_pred)             # 予測結果を保存する
        else:
            # 学習を行う
            x = Datum({"data": data[i]})     # 1期前のデータを入力にする
            regression.train([[y_true, x]])  # 学習する
            prediction.append(y_true)        # 正解を保存する(可視化用)

    # RMSEを計算
    rmse = np.sqrt(((np.array(answer[n_train:]) - np.array(prediction[n_train:]))**2).mean())
    
    # 可視化
    df_result = pd.DataFrame({"answer": answer, "predict": prediction}, index=index)
    df_result[int(len(df_result)*(1.0-disp_rate)):].plot(figsize=(18,12), title="RMSE={0:.3f}".format(rmse))

### Step.4 実験：ノッティンガムの平均気温予測

In [ ]:
# データセットの取得
target = 'nottem'
df = sm.datasets.get_rdataset(target).data
def convert_date(date):
    month, year = np.modf(float(date))
    year = round(year)
    month = round(month / (1.0 / 12) + 1)
    return datetime.datetime(int(year), int(month), 1)
df.time = df.time.apply(convert_date)
df.index = pd.to_datetime(df.time, format='%Y')
df[target] = df[target].astype(float)
df = df.drop('time', axis=1)

# pandas形式をlistに変換する
data = df[target].as_matrix().tolist()

# 学習と予測
train_and_estimate(data, disp_rate=0.3)

### Step.4 実験：太陽黒点予測

In [ ]:
# データセットの取得
target = 'sunspots'
df = sm.datasets.get_rdataset(target).data
# 日付変換 + 整形
def convert_date(date):
    month, year = np.modf(float(date))
    year = round(year)
    month = round(month / (1.0 / 12) + 1)
    return datetime.datetime(int(year), int(month), 1)

df.time = df.time.apply(convert_date)
df.index = pd.to_datetime(df.time, format='%Y')
df[target] = df[target].astype(float)
df = df.drop('time', axis=1)

# pandas形式をlistに変換する
data = df[target].as_matrix().tolist()

# 学習と予測
train_and_estimate(data, disp_rate=0.3)